In [1]:
#!/env/bin/python

import init

import findspark
findspark.init()

import os
import pandas as pd
import numpy as np
from pyspark.sql import *
from pyspark import *
import pyspark.sql.functions as f
from timeit import default_timer as timer
import logging
from pyrecdp.data_processor import *
from pyrecdp.utils import *
from pyspark.sql.functions import pandas_udf, PandasUDFType

path_prefix = "hdfs://"
current_path = "/recsys2021_0608_scalaudf_test/"
original_folder = "/recsys2021_0608/"

scala_udf_jars = "/mnt/nvme2/chendi/BlueWhale/recdp/ScalaProcessUtils/target/core-0.1.0-jar-with-dependencies.jar"

##### 1. Start spark and initialize data processor #####
t0 = timer()
spark = SparkSession.builder.master('yarn')\
    .appName("scala_udf_column")\
    .config("spark.sql.broadcastTimeout", "7200")\
    .config("spark.cleaner.periodicGC.interval", "10min")\
    .config("spark.driver.extraClassPath", f"{scala_udf_jars}")\
    .config("spark.executor.extraClassPath", f"{scala_udf_jars}")\
    .config("spark.executor.memory", "20g")\
    .config("spark.executor.memoryOverhead", "6g")\
    .getOrCreate()



/mnt/nvme2/chendi/BlueWhale/recdp


In [3]:
# def process_data(df, proc, output_name):
#     proc.spark.udf.registerJavaFunction("dummy","com.intel.recdp.SortStringArrayByFrequency")
#     op_feature_add = FeatureAdd(cols={'tweet': "f.expr('dummy(text_tokens)')"}, op='inline')
# 
#     # execute
#     proc.reset_ops([op_feature_add])
#     t1 = timer()
#     df = proc.transform(df, name=output_name, df_cnt=626242930)
#     t2 = timer()
#     print("process_data decode took %.3f" % (t2 - t1))
# 
#     return df


# 1.1 prepare dataFrames
# 1.2 create RecDP DataProcessor
# proc = DataProcessor(spark, path_prefix,
#                      current_path=current_path, shuffle_disk_capacity="1200GB")

gateway = spark.sparkContext._gateway
spark._jsparkSession.udf().register("CodegenSeparator", gateway.jvm.org.apache.spark.sql.api.CodegenSeparator())
spark._jsparkSession.udf().register("CodegenSeparator0", gateway.jvm.org.apache.spark.sql.api.CodegenSeparator0())
spark._jsparkSession.udf().register("CodegenSeparator1", gateway.jvm.org.apache.spark.sql.api.CodegenSeparator1())
spark._jsparkSession.udf().register("CodegenSeparator2", gateway.jvm.org.apache.spark.sql.api.CodegenSeparator2())
df = spark.read.parquet(path_prefix + original_folder)
df = df.select("tweet_timestamp")
df = df.withColumn('tweet_timestamp', f.expr("CodegenSeparator2(tweet_timestamp)"))
#df = df.withColumn('CodeSeparator', f.expr("CodegenSeparator()"))
print(df.dtypes)
df.show()

# ===============================================
# decode tweet_tokens
#df = process_data(df, proc, output_name="scala_udf_test")

[('tweet_timestamp', 'bigint')]
+---------------+
|tweet_timestamp|
+---------------+
|     1613574302|
|     1612628499|
|     1613233141|
|     1613373619|
|     1614053869|
|     1613130630|
|     1613721703|
|     1612706347|
|     1613905759|
|     1613854589|
|     1613217462|
|     1612584698|
|     1613590692|
|     1613805413|
|     1612540790|
|     1613509274|
|     1613437436|
|     1614099587|
|     1612914478|
|     1612815075|
+---------------+
only showing top 20 rows



In [ ]:
def process_data(df, proc, output_name):
    proc.spark.udf.registerJavaFunction("dummy","com.intel.recdp.SortStringArrayByFrequency")
    op_feature_add = FeatureAdd(cols={'tweet': "f.expr('dummy(text_tokens)')"}, op='inline')

    # execute
    proc.reset_ops([op_feature_add])
    t1 = timer()
    df = proc.transform(df, name=output_name, df_cnt=626242930)
    t2 = timer()
    print("process_data decode took %.3f" % (t2 - t1))

    return df


# 1.1 prepare dataFrames
# 1.2 create RecDP DataProcessor
proc = DataProcessor(spark, path_prefix,
                     current_path=current_path, shuffle_disk_capacity="1200GB")
spark.udf.registerJavaFunction("sortStringArrayByFrequency","com.intel.recdp.SortStringArrayByFrequency")

df = spark.read.parquet(path_prefix + original_folder)

df = df.select("reply_timestamp", 'tweet_id').groupby('tweet_id').agg(f.collect_list('reply_timestamp').alias('reply_timestamp'))
print(df.dtypes)
df = df.withColumn('sorted_reply_timestamp', f.expr('sortStringArrayByFrequency(reply_timestamp)'))
print(df.dtypes)

df.show()

# ===============================================
# decode tweet_tokens
#df = process_data(df, proc, output_name="scala_udf_test")

In [ ]:
df.show()